In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("Data/data.csv")
data

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,18,1,0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
2388,17,0,0,1,7.583217,4,1,4,0,1,0,0,3.279150,4.0
2389,16,1,0,2,6.805500,20,0,2,0,0,0,1,1.142333,2.0
2390,16,1,1,0,12.416653,17,0,2,0,1,1,0,1.803297,1.0


In [3]:
inputs = data.iloc[0, :13].to_dict()
inputs

{'Age': 17.0,
 'Gender': 1.0,
 'Ethnicity': 0.0,
 'ParentalEducation': 2.0,
 'StudyTimeWeekly': 19.83372280785472,
 'Absences': 7.0,
 'Tutoring': 1.0,
 'ParentalSupport': 2.0,
 'Extracurricular': 0.0,
 'Sports': 0.0,
 'Music': 1.0,
 'Volunteering': 0.0,
 'GPA': 2.929195591667681}

In [4]:
import tensorflow as tf
import base64
import json
import requests

def prepare_json(inputs: dict):
    feature_spec = {}

    # Define mapping from input keys to TensorFlow Feature types
    feature_mapping = {
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Age'])])),
        "Gender": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Gender'])])),
        "Ethnicity": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Ethnicity'])])),
        "ParentalEducation": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['ParentalEducation'])])),
        "StudyTimeWeekly": tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs['StudyTimeWeekly'])])),
        "Absences": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Absences'])])),
        "Tutoring": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Tutoring'])])),
        "ParentalSupport": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['ParentalSupport'])])),
        "Extracurricular": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Extracurricular'])])),
        "Sports": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Sports'])])),
        "Music": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Music'])])),
        "Volunteering": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Volunteering'])])),
        "GPA": tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs['GPA'])])),
    }

    # Create TensorFlow Example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_mapping)
    ).SerializeToString()

    # Encode Example to base64
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })

json_data = prepare_json(inputs)

endpoint = "http://localhost:8080/v1/models/grade-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    # Assuming prediction is a list of lists with each list containing probabilities for each class
    max_prob_index = prediction[0].index(max(prediction[0]))  # Get the index of the highest probability
    if max_prob_index == 0:
        result = "Label 0.0"
    elif max_prob_index == 1:
        result = "Label 1.0"
    elif max_prob_index == 2:
        result = "Label 2.0"
    elif max_prob_index == 3:
        result = "Label 3.0"
    elif max_prob_index == 4:
        result = "Label 4.0"
    else:
        result = "Unknown label"
else:
    result = "Error: No predictions found in response."

print(result)


Label 2.0
